In [9]:
import bioMass.dataViz as dv
from bioMass.dataloader import SentinelDataset, SentinelDataModule
from bioMass.moran_Dataloader import MoranSentinelDataset
from bioMass.model import Unet, DoubleStreamUnet
from pytorch_lightning import Trainer
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, random_split
from tqdm.notebook import tqdm
from torch import nn
import os
import numpy as np
import pandas as pd
from glob import glob
import yaml

from pytorch_lightning import loggers as pl_loggers

%config Completer.use_jedi = False
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# MPS Docs
# https://pytorch.org/docs/master/notes/mps.html

if torch.backends.mps.is_available(): # Mac M1/M2
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

loader_device = torch.device('cpu')  # found that using cpu for data loading was faster than gpu (for my device)
print(f'training device: {device}')
print(f'loader_device: {loader_device}')

dir_tiles = '/home/ubuntu/Thesis/backup_data/bioMass_data/train_features/'
dir_target = '/home/ubuntu/Thesis/backup_data/bioMass_data/train_agbm/'
dir_saved_models = './trained_models'

training device: cuda
loader_device: cpu


In [11]:
# this file specifies which month of data to use for training for each chipid 
# See the preprocesing notebook for an example of producing this  
tile_file = '/home/ubuntu/Thesis/backup_data/bioMass_data/TILE_LIST_BEST_MONTHS.csv'

max_chips = None # number of chips to use from training set, None = Use All  

# A custom dataloader for Sentinel data 
dataset = MoranSentinelDataset(tile_file=tile_file, 
                             dir_tiles=dir_tiles, 
                             dir_target=dir_target,
                             max_chips=max_chips,
                             transform=None,
                             device=loader_device
                            )

In [12]:
dataset[0]['label'].shape

torch.Size([1, 256, 256])

In [13]:
dataset[0]['image_s2'].shape

torch.Size([11, 256, 256])